In [1]:
#https://keon.io/deep-q-learning/
#에서 참고


C:\Users\sh2\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam
from collections import deque
import numpy as np
import gym
import random

In [85]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen =2000)
        self.gamma = 0.95 # 보상이 느릴수록 페널티를 주기위해
        self.epsilon = 1.0 #e
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.target_model = self.build_model()
        self.main_model = self.build_model()
        
    def build_model(self):
        model = Sequential()
        model.add(Dense(128,input_dim=self.state_size,activation='relu'))
        model.add(Dense(128, activation = 'relu'))
        model.add(Dense(self.action_size, activation = 'softmax'))
        model.compile(loss='mse' , optimizer = Adam(lr=self.learning_rate))
        return model
    def remember(self, state, action, rewar, next_state, done):
        self.memory.append((state,action,reward,next_state,done))
    
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            
            return random.randrange(self.action_size)
        
        act_values = self.main_model.predict(state)
        return np.argmax(act_values[0])
    
    def replay(self, batch_size):
        #memory 의 길이보다 batch_size 가 작게 들어온 경우 return
        x_stack = np.empty(0).reshape(0,self.state_size)
        y_stack = np.empty(0).reshape(0,self.action_size)
        if(len(self.memory) <= batch_size):
            return
        minibatch = random.sample(self.memory, batch_size)

        #target_model
        #main_model
        for state, action, reward, next_state, done in minibatch:
            Q = self.main_model.predict(state)
            
            if done:
                Q[0,action] = reward
                
            else :
                Q[0,action] = reward + self.gamma * np.max(self.target_model.predict(next_state))
            
           
            
            y_stack = np.vstack([y_stack,Q])
            x_stack = np.vstack([x_stack,state])
            
        

        self.main_model.fit(x_stack,y_stack)
        

                


In [86]:
env = gym.make('CartPole-v0')

    
agent = DQNAgent(4,2)

In [ ]:

episodes = 1000
# Iterate the game
for e in range(episodes):
    # reset state in the beginning of each game
    state = env.reset()
        
    state = np.reshape(state, [1, 4])
      
        # time_t represents each frame of the game
        # Our goal is to keep the pole upright as long as possible until score of 500
        # the more time_t the more score
    if agent.epsilon > agent.epsilon_min:
            agent.epsilon *= agent.epsilon_decay    
    for time_t in range(10001):
            # turn this on if you want to render
            #env.render()
            # Decide action
        
        action = agent.act(state)
            # Advance the game to the next frame based on the action.
            # Reward is 1 for every frame the pole survived
        next_state, reward, done, _ = env.step(action)
        next_state = np.reshape(next_state, [1, 4])
            # Remember the previous state, action, reward, and done
        if done:
            reward = -10
        agent.remember(state, action, reward, next_state, done)
        
        state = next_state
        
        if done:
                
            print("episode: {}/{}, score: {}"
                  .format(e, episodes, time_t))
            break
    
    if e % 10 == 1:
        for _ in range(50):
            agent.replay(24)
    agent.target_model.set_weights(agent.main_model.get_weights())    

episode: 0/1000, score: 25
episode: 1/1000, score: 13
Epoch 1/1
24/24 [==============================] - 1s 35ms/step - loss: 2.7515
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 2.7345
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 2.7224
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 2.7232
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 0.4772
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 2.6817
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 2.6690
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 2.7010
Epoch 1/1
24/24 [==============================] - 0s 42us/step - loss: 2.6362
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 0.4777
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 4.7972
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 2.644

episode: 12/1000, score: 34
episode: 13/1000, score: 50
episode: 14/1000, score: 20
episode: 15/1000, score: 18
episode: 16/1000, score: 11
episode: 17/1000, score: 28
episode: 18/1000, score: 16
episode: 19/1000, score: 9
episode: 20/1000, score: 23
episode: 21/1000, score: 28
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 5.2321
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 2.8277
Epoch 1/1
24/24 [==============================] - 0s 125us/step - loss: 0.6377
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 2.7635
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 2.8633
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 0.8019
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 0.6938
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 5.3124
Epoch 1/1
24/24 [==============================] - 0s 166us/step - loss: 4.8946
Epoch 1/

Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 5.0477
Epoch 1/1
24/24 [==============================] - 0s 42us/step - loss: 4.8327
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 3.0047
episode: 32/1000, score: 18
episode: 33/1000, score: 58
episode: 34/1000, score: 29
episode: 35/1000, score: 31
episode: 36/1000, score: 48
episode: 37/1000, score: 13
episode: 38/1000, score: 22
episode: 39/1000, score: 19
episode: 40/1000, score: 15
episode: 41/1000, score: 11
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 0.5512
Epoch 1/1
24/24 [==============================] - 0s 125us/step - loss: 2.9471
Epoch 1/1
24/24 [==============================] - 0s 125us/step - loss: 0.5259
Epoch 1/1
24/24 [==============================] - 0s 42us/step - loss: 0.5874
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 2.7647
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 4.8305
Epoch 1

Epoch 1/1
24/24 [==============================] - 0s 42us/step - loss: 0.6184
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 2.6715
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 4.9114
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 0.5084
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 0.5480
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 0.6454
episode: 52/1000, score: 26
episode: 53/1000, score: 20
episode: 54/1000, score: 20
episode: 55/1000, score: 15
episode: 56/1000, score: 17
episode: 57/1000, score: 19
episode: 58/1000, score: 31
episode: 59/1000, score: 29
episode: 60/1000, score: 20
episode: 61/1000, score: 22
Epoch 1/1
24/24 [==============================] - 0s 125us/step - loss: 6.8803
Epoch 1/1
24/24 [==============================] - 0s 166us/step - loss: 2.7046
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 0.6225
Epoch 1

Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 2.8291
Epoch 1/1
24/24 [==============================] - 0s 125us/step - loss: 2.9894
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 5.3194
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 2.8562
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 0.6144
Epoch 1/1
24/24 [==============================] - 0s 125us/step - loss: 2.7343
Epoch 1/1
24/24 [==============================] - 0s 166us/step - loss: 0.7465
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 2.7617
Epoch 1/1
24/24 [==============================] - 0s 125us/step - loss: 0.7267
episode: 72/1000, score: 17
episode: 73/1000, score: 18
episode: 74/1000, score: 19
episode: 75/1000, score: 31
episode: 76/1000, score: 15
episode: 77/1000, score: 12
episode: 78/1000, score: 41
episode: 79/1000, score: 14
episode: 80/1000, score: 11
episode: 81/1000, score: 22
Epoch

...

Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 7.5418
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 3.0232
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 2.7807
Epoch 1/1
24/24 [==============================] - 0s 125us/step - loss: 0.5132
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 2.6671
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 7.4751
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 3.0911
Epoch 1/1
24/24 [==============================] - 0s 42us/step - loss: 2.6938
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 7.0788
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 5.4909
episode: 362/1000, score: 13
episode: 363/1000, score: 17
episode: 364/1000, score: 14
episode: 365/1000, score: 19
episode: 366/1000, score: 16
episode: 367/1000, score: 12
episode: 368/1000, score: 16
episod

Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 9.7950
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 7.5044
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 5.2439
Epoch 1/1
24/24 [==============================] - 0s 125us/step - loss: 2.6383
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 9.6615
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 2.9413
Epoch 1/1
24/24 [==============================] - 0s 125us/step - loss: 0.5978
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 5.6102
Epoch 1/1
24/24 [==============================] - 0s 125us/step - loss: 7.3167
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 0.6405
Epoch 1/1
24/24 [==============================] - 0s 208us/step - loss: 2.5621
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 2.9872
Epoch 1/1
24/24 [==============================]

Epoch 1/1
24/24 [==============================] - 0s 125us/step - loss: 7.6195
Epoch 1/1
24/24 [==============================] - 0s 249us/step - loss: 2.7034
Epoch 1/1
24/24 [==============================] - 0s 166us/step - loss: 0.7593
Epoch 1/1
24/24 [==============================] - 0s 125us/step - loss: 5.0374
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 2.9533
Epoch 1/1
24/24 [==============================] - 0s 208us/step - loss: 0.5625
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 5.1145
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 5.5611
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 5.4980
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 0.5687
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 5.4262
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 2.7127
Epoch 1/1
24/24 [==============================

Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 7.5657
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 5.1726
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 2.8470
Epoch 1/1
24/24 [==============================] - 0s 125us/step - loss: 0.5391
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 0.7057
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 2.9264
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 5.4032
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 2.9026
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 0.5848
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 2.9210
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 0.7576
Epoch 1/1
24/24 [==============================] - 0s 125us/step - loss: 0.6253
Epoch 1/1
24/24 [==============================] -

Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 0.5889
Epoch 1/1
24/24 [==============================] - 0s 125us/step - loss: 9.5727
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 11.8399
Epoch 1/1
24/24 [==============================] - 0s 42us/step - loss: 7.5926
Epoch 1/1
24/24 [==============================] - 0s 125us/step - loss: 5.0301
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 4.9843
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 9.8266
Epoch 1/1
24/24 [==============================] - 0s 41us/step - loss: 5.1679
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 3.0150
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 5.0083
Epoch 1/1
24/24 [==============================] - 0s 83us/step - loss: 0.6161
Epoch 1/1
24/24 [==============================] - 0s 92us/step - loss: 2.7469
Epoch 1/1
24/24 [==============================] 